In [ ]:
import numpy as np
import time
from keras.preprocessing.image import save_img
from keras.applications import VGG16
from keras import backend as K
from keras.models import Model


from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
img_width, img_height = 128, 128

In [ ]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
layer_dict = dict([(layer.name, layer) for layer in base_model.layers])
base_model.summary()

In [ ]:
img = image.load_img('horse.jpg', target_size=(img_width, img_height))
img = image.img_to_array(img)
img_ = np.expand_dims(img, 0) 
img = preprocess_input(img_)
print(img.shape)

In [ ]:
feature = base_model.predict(img)
input_img = base_model.layers[0].input

In [ ]:
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

In [ ]:
def get_filter(layer_name):
    
    layer_pool_extractor = Model(inputs=base_model.input, outputs=base_model.get_layer(layer_name).output)
    feature_maps = layer_pool_extractor.predict(img_)
    
    _, height, width, depth = feature_maps.shape

    kept_filters = []
    
    #more depth = greater computational time
    for filter_index in range(depth):
        print('Processing filter {} of {} filters'.format((filter_index+1), depth), end="")
        
        start_time = time.time()
        
        layer_output = layer_dict[layer_name].output
        
        if K.image_data_format() == 'channels_first':
            loss = K.mean(layer_output[:, filter_index, :, :])
        else:
            loss = K.mean(layer_output[:, :, :, filter_index])

        #gradient of the input picture wrt this loss
        grads = K.gradients(loss, input_img)[0]

        #normalize the gradient
        grads = normalize(grads)

        #function returns the loss and grads given the input picture
        iterate = K.function([input_img], [loss, grads])
        
        # step size for gradient ascent
        step = 1.

        # we start from a gray image with some random noise
        if K.image_data_format() == 'channels_first':
            input_img_data = np.random.random((1, 3, img_width, img_height))
        else:
            input_img_data = np.random.random((1, img_width, img_height, 3))
        input_img_data = (input_img_data - 0.5) * 20 + 128

        # we run gradient ascent for 20 steps
        for i in range(20):
            loss_value, grads_value = iterate([input_img_data])
            input_img_data += grads_value * step

            if loss_value <= 0.:
                # some filters get stuck to 0, we can skip them
                break

        # decode the resulting input image
        if loss_value > 0:
            img = deprocess_image(input_img_data[0])
            kept_filters.append((img, loss_value))
            
        end_time = time.time()
        print(' in %ds' % (end_time - start_time))

    return kept_filters

In [ ]:
def plot_filter(kept_filters):
    n = np.int(np.floor(np.sqrt(np.shape(kept_filters)[0])))
    
    # the filters that have the highest loss are assumed to be better-looking.
    kept_filters.sort(key=lambda x: x[1], reverse=True)
    kept_filters = kept_filters[:n * n]

    margin = 5
    width = n * img_width + (n - 1) * margin
    height = n * img_height + (n - 1) * margin
    stitched_filters = np.zeros((width, height, 3))

    # fill the picture with our saved filters
    for i in range(n):
        for j in range(n):
            img, loss = kept_filters[i * n + j]
            stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                             (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img

    # save the result to disk
    print('Done')
    save_img('stitched_filters_%dx%d.png' % (n, n), stitched_filters)

In [ ]:
layername = 'block4_conv3'
filter_return = get_filter(layer_name=layername)

In [ ]:
plot_filter(kept_filters=filter_return)